ปฏิบัติการครั้งที่ 5 กระบวนวิชา 229351 Statistical Learning for Data Science

คำชี้แจง

ให้เริ่มทำปฏิบัติการจาก colab notebook ที่กำหนดให้ จากนั้น share แล้วส่ง link มาใน mango.cmu.ac.th

ดาวน์โหลดข้อมูลรถยนต์ชนิดต่างใน link ข้างล่างนี้
http://www.donlapark.cmustat.com/229351/data/Auto.csv
หรือรันโค้ดข้างล่าง

In [1]:
!wget http://www.donlapark.cmustat.com/229351/data/Auto.csv  

--2023-01-05 11:05:09--  http://www.donlapark.cmustat.com/229351/data/Auto.csv
Resolving www.donlapark.cmustat.com (www.donlapark.cmustat.com)... 150.107.31.67
Connecting to www.donlapark.cmustat.com (www.donlapark.cmustat.com)|150.107.31.67|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://donlapark.pages.dev/229351/data/Auto.csv [following]
--2023-01-05 11:05:09--  https://donlapark.pages.dev/229351/data/Auto.csv
Resolving donlapark.pages.dev (donlapark.pages.dev)... 172.66.47.56, 172.66.44.200, 2606:4700:310c::ac42:2f38, ...
Connecting to donlapark.pages.dev (donlapark.pages.dev)|172.66.47.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18079 (18K) [text/csv]
Saving to: ‘Auto.csv.2’

Auto.csv.2          100%[===================>]  17.66K  --.-KB/s    in 0s      

2023-01-05 11:05:09 (50.0 MB/s) - ‘Auto.csv.2’ saved [18079/18079]



In [2]:
# import module ที่ต้องใช้

import numpy as np
import pandas as pd
from scipy import stats

In [3]:
# อ่านไฟล์ csv ก่อนเพื่อหา missing values
auto_df = pd.read_csv('Auto.csv',na_values=["?"])
predictor_names = auto_df.columns

# ลบแถวที่มี missing values
auto_df = auto_df.dropna()
auto_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


### ตัวแปรต่าง ๆ มีดังนี้
mpg

    miles per gallon
cylinders

    Number of cylinders between 4 and 8
displacement

    Engine displacement (cu. inches)
horsepower

    Engine horsepower
weight

    Vehicle weight (lbs.)
acceleration

    Time to accelerate from 0 to 60 mph (sec.)
year

    Model year (modulo 100)
origin

    Origin of car (1. American, 2. European, 3. Japanese)
name

    Vehicle name


In [4]:
auto_df = auto_df.drop(['name', 'origin'],axis=1)

auto_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year
0,18.0,8,307.0,130.0,3504,12.0,70
1,15.0,8,350.0,165.0,3693,11.5,70
2,18.0,8,318.0,150.0,3436,11.0,70
3,16.0,8,304.0,150.0,3433,12.0,70
4,17.0,8,302.0,140.0,3449,10.5,70


## 1. Multivariate linear regression

เปลี่ยน $\texttt{dataframe}$ ให้เป็น numpy array ด้วย $\texttt{dataframe.to_numpy()}$  

1. Response: $\texttt{y}$ เป็นคอลัมน์เวกเตอร์ของ $\texttt{mpg}$
2. Predictors: $\texttt{X}$ เป็นเมทริกซ์ของตัวแปรที่เหลือ (ใช้ $\texttt{auto_df.drop}$)


In [5]:
y = auto_df['mpg'].to_numpy()
X = auto_df.drop(['mpg'],axis=1).to_numpy()

คอลัมน์แรกของ $\texttt{X}$ ต้องเป็น $(1, 1, \ldots ,1 )^T$ (จำนวนเลข 1 เท่ากับจำนวนแถวของ $\texttt{X}$)  
สร้างเวกเตอร์ที่มีแต่เลข 1 และมีความยาวที่เหมาะสมด้วยคำสั่งใดคำสั่งหนึ่งต่อไปนี้  
1. $\texttt{[1]*X.shape[0]}$  
2. $\texttt{np.ones(X.shape[0])}$  

In [6]:
X.shape

(392, 6)

In [8]:
ones =  [1]*X.shape[0] #your answer here

# ใส่ ones ให้เป็นคอลัมน์แรกของ X
X = np.c_[ones,X]

In [9]:
#เช็คคำตอบ
print(X)

[[1.000e+00 8.000e+00 3.070e+02 ... 3.504e+03 1.200e+01 7.000e+01]
 [1.000e+00 8.000e+00 3.500e+02 ... 3.693e+03 1.150e+01 7.000e+01]
 [1.000e+00 8.000e+00 3.180e+02 ... 3.436e+03 1.100e+01 7.000e+01]
 ...
 [1.000e+00 4.000e+00 1.350e+02 ... 2.295e+03 1.160e+01 8.200e+01]
 [1.000e+00 4.000e+00 1.200e+02 ... 2.625e+03 1.860e+01 8.200e+01]
 [1.000e+00 4.000e+00 1.190e+02 ... 2.720e+03 1.940e+01 8.200e+01]]


### Exercise 1: จงเขียนฟังก์ชันต่อไปนี้ 
1. `linear_model` มี input เป็นแมทริกซ์ `X` เวกเตอร์ `y` และ output เป็นเวกเตอร์ของสัมประสิทธิ์ 
2. `predict` มี input เป็นแมทริกซ์ `X` เวกเตอร์ของสัมประสิทธิ์ `beta` และ output เป็นเวกเตอร์ของค่าทำนาย `y_pred` 

1. Matrix multiplication $AB = \texttt{A@B}$
2. Transpose: $X^T = \texttt{X.T}$
3. Inverse: $X^{-1} = \texttt{np.linalg.inv(X)}$

$$ \hat{\beta} = (X^TX)^{-1}X^Ty $$

In [ ]:
# Let's implement linear regression from scratch
# using the formula provided in the class

def linear_model(X, y):
    """X: เมทริกซ์ของตัวแปรต้น"""
    """y: เวกเตอร์ของตัวแปรตาม"""
    """Return: เวกเตอร์ของ parameter beta จากการสร้าง linear regressions model
    ด้วย Ordinary Least Squares (OLS)"""
    # TODO: your code here


    return beta 


def predict(X, beta):
    """beta: array ของสัมประสิทธิ์"""
    """X: เมทริกซ์ของตัวแปรต้น""" 
    """Return: ค่าทำนาย y_pred = X*beta """
    # TODO: your code here


    return y_pred

In [ ]:
beta = linear_model(X, y)
y_pred = predict(X, beta)

beta

#### จงหา $n$ (จำนวนแถวของ $\texttt{X}$) และ $p$ (จำนวนตัวแปร) ด้วย method $\texttt{X.shape[...]}$

In [ ]:
n = X.shape[0]

p = X.shape[1]-1

print('Number of observations: ',n)
print('Number of variables: ',p)

## 2. F-statistic

### Exercise 2: จงเขียนสมมติฐานหลังและสมมติฐานแย้งที่ใช้ F-statistic ในการทดสอบ

### Exercise 3: จากเวกเตอร์ของ outcomes `y` จงคำนวณ Total Sum of Squares (TSS)

$$ \text{TSS} = \sum_{i=1}^n (y_i-\bar{y})^2 $$
โดยใช้คำสั่งต่อไปนี้

$\texttt{np.sum}([y_1,y_2,...,y_n])=y_1+y_2+...+y_n$

$\texttt{np.square}([y_1,y_2,...,y_n])=[y_1^2,y^2_2,...,y^2_n]$

In [ ]:
# Calculate Total Sum of Squares
y_mean = y.mean()

TSS = # TODO: your code here

RSS = np.sum(np.square(y-y_pred))

คำนวณ $F$-statistic ด้วย
$$F_{p,n-p-1} = \frac{(\text{TSS}-\text{RSS})/p}{\text{RSS}/(n-p-1)}$$
หา $n$ และจำนวนของตัวแปร $p$ ด้วย $\texttt{X.shape}[\cdot]$

### Exercise 4: คำนวณค่า F

In [ ]:
F = 

print(F)

In [ ]:
# Compute p_value of the F-statistic

p_value = stats.f.sf(F, p, n-p-1)

print(p_value)

### Exercise 5: จากการทดสอบสมมติฐานที่นัยสำคัญ 0.05 เราได้ข้อสรุปอย่างไรกับตัวแปรต้น (predictors) จากค่า p-value ที่ได้ข้างบน

## 3. คำนวณ $t$-statistic

#### ขั้นตอนในการคำนวณ $t$-statistic

\begin{align*} 
\text{RSE} &= \sqrt{\frac{\text{RSS}}{n-p-1}} \\
\text{SE}(\hat{\beta}) &=\text{RSE}\cdot\sqrt{\text{diag}\left( (X^TX)^{-1}\right)} \\
t &= \frac{\hat{\beta}}{\text{SE}(\hat{\beta})}
\end{align*}

### Exercise 6: จงใช้ฟังก์ชัน `t_to_p_values` ข้างล่างในการหาว่ามีตัวแปรใดบ้างที่มีค่า p-value น้อยกว่า 0.05 **แล้วระบุตัวแปรทั้งหมดใน text block ข้างล่าง**  

#### ใช้คำสั่ง `A.diagonal()` เพื่อคำนวณเวกเตอร์ของค่าในแนวทแยงของ `A`

In [ ]:
# TODO: your code here

RSE = 
SE =
t =

In [ ]:
def t_to_p_values(t_statistics):
    """t_statistics: numpy array ของค่า t-statistic ของตัวแปรต่างๆ
    Return: เวกเตอร์ของค่า p-value ของค่า t-statistic ที่ให้มา"""
    return stats.t.sf(np.abs(t_statistics), n-p-1)*2

# Standard error of each variable in X
p = t_to_p_values(t)

In [12]:
print("List of predictors:", predictor_names[1:-2])

List of predictors: Index(['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration',
       'year'],
      dtype='object')


### คำตอบของ Exercise 6:





### Exercise 7: จากการทดสอบสมมติฐานที่นัยสำคัญ 0.05 มีตัวแปรต้นตัวใดบ้างที่ไม่น่าจะมีความสำคัญในโมเดล linear regression นี้

### คำตอบของ Exercise 7:





#### statsmodels library สามารถคำนวณค่าต่างๆ เหล่านี้ได้ ผลที่แสดงจะคล้ายกับใน R  
#### มีสองวิธึในการทำ linear regression: $\texttt{statsmodels.api.sm.OLS}$ 
#### และ $\texttt{statsmodels.formula.api.smf.ols}$

In [10]:
# The statsmodels library provides a convenient means to get the
# same statistics
import statsmodels.api as sm

X = auto_df.drop(['mpg'], axis=1)
X = sm.add_constant(X)     # add bias constant
y = auto_df['mpg']

# add bias constant; 
#without this the equation turns into y = βx

# syntax is OLS(response, predictor)
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.809
Model:                            OLS   Adj. R-squared:                  0.806
Method:                 Least Squares   F-statistic:                     272.2
Date:                Thu, 05 Jan 2023   Prob (F-statistic):          3.79e-135
Time:                        11:18:31   Log-Likelihood:                -1036.5
No. Observations:                 392   AIC:                             2087.
Df Residuals:                     385   BIC:                             2115.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -14.5353      4.764     -3.051   

In [15]:
1.0e1  # 1.0x10

10.0

In [14]:
1.0e-1  # 1.0x0.1

0.1

In [16]:
1.0e-2  # 1.0x0.01

0.01